In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir
from sklearn.preprocessing import scale
#from pyclustertend import hopkins, vat, ivat
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
from avgn.clusterability.hopkins import hopkins_statistic

In [4]:
datasets = list(DATA_DIR.glob('embeddings/**/*.pickle'))
len(datasets)

296

In [5]:
datasets = [i for i in datasets if i.parent.stem not in [
    'buckeye',
    'BIRD_DB_Vireo_cassinii',
    'swamp_sparrow',
    'batsong_segmented'
]]

In [6]:
len(datasets)

291

In [7]:
def remove_outliers(z, pct = 99.5):
    """ GPU based UMAP algorithm produces some outliers that UMAP does not, but is much faster
    this is a quick fix for that. 
    """
    _min = np.percentile(z, (100-pct), axis=0)
    _max = np.percentile(z, pct, axis=0)
    for col in range(np.shape(z)[1]):
        mask = z[:,col] < _min[col]
        z[mask,col] = _min[col]
        mask = z[:,col] > _max[col]
        z[mask,col] = _max[col]
    return z

In [8]:
ensure_dir(DATA_DIR / 'clusterability' / 'convex_sample_indvs')

In [9]:
def gen_clusterability_df(dataset):
    """
    m_prop_n is the number of samples over X to perform hopkins statistic on (.1 is reccomended)
    """
    save_loc = DATA_DIR / 'clusterability' / 'convex_sample_indvs' / \
        (dataset.parent.stem + '_ ' + dataset.stem + '.pickle')

    #if save_loc.exists():
    #    return

    ds = pd.read_pickle(dataset)
    specs = np.stack(ds[['spectrogram']].spectrogram.values)
    specs = specs.reshape(len(specs), -1)
    specs = scale(specs)

    umap_proj = np.vstack(ds[['umap']].umap.values)
    umap_proj = remove_outliers(umap_proj, pct=99.5)
    umap_proj = scale(umap_proj)
    
    nex = len(umap_proj)

    print((dataset, np.shape(umap_proj)))
    fig, ax = plt.subplots()
    ax.scatter(umap_proj[:, 0], umap_proj[:, 1], s=1, color='k', alpha=0.1)
    plt.show()

    hopkins_dict = {
        'umap':
        {
            0.01: hopkins_statistic(umap_proj, m_prop_n=0.01, n_neighbors=1, distribution="uniform_convex_hull"),
            0.1: hopkins_statistic(umap_proj, m_prop_n=0.1, n_neighbors=1, distribution="uniform_convex_hull"),
        },

    }
    
    dsname =  dataset.parent.parent.stem if dataset.parent.stem == 'indvs' else dataset.parent.stem

    clusterability_df = pd.DataFrame([[dataset, dsname, dataset.stem,
                                       hopkins_dict['umap'][0.01], hopkins_dict['umap'][0.1], nex]], columns=[
        'df', 'dataset', 'indv',
        'umap_hopkins_1', 'umap_hopkins_10', 'nex'
    ])

    clusterability_df.to_pickle(save_loc)

In [10]:
ensure_dir(DATA_DIR / 'clusterability' / 'convex_sample')

In [11]:
from joblib import Parallel, delayed

In [12]:
clust_data = Parallel(n_jobs=-1, verbose=10)(
    delayed(gen_clusterability_df)(dataset)
    for dataset in tqdm(datasets)
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:   12.7s
/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the execu

[Parallel(n_jobs=-1)]: Done 258 out of 291 | elapsed:   19.4s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done 288 out of 291 | elapsed:   26.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 291 out of 291 | elapsed:   29.0s finished
